<a href="https://colab.research.google.com/github/SteelworkerDB/AML_W2/blob/main/IngCaracteristicas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#EJERCICIO DE INGENIERÍA DE CARACTERÍSTICAS

In [ ]:
import pandas as pd
import numpy as np


In [ ]:
#importar sklearn para modelado
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_selector
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

In [ ]:
#CARGA DE DATOS
filename='/content/bikeshare_train - bikeshare_train.csv'
df=pd.read_csv(filename)
df.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 0:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 1:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 2:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 3:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 4:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


OBJETIVO: COUNT (NUMERO DE BICICLETAS ALQUILADAS EN ESE DÍA)

1. Elimina columnas "casual" y "registered"

In [ ]:
df=df.drop(columns=['casual','registered'])

2. Transforma columna de fecha y hora en un tipo de fecha y hora, agrega 3 columnas de mes, día de la semana, hora del día

In [ ]:
df['datetime'].dtype

dtype('O')

In [ ]:
df['datetime']=pd.to_datetime(df['datetime'])

In [ ]:
df['datetime'].dtype

dtype('<M8[ns]')

In [ ]:
df['mes']=df['datetime'].dt.month_name()
df['weekday']=df['datetime'].dt.day_name()
df['hora']=df['datetime'].dt.time
df=df.drop(columns=['datetime','season'])

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   holiday     10886 non-null  int64  
 1   workingday  10886 non-null  int64  
 2   weather     10886 non-null  int64  
 3   temp        10886 non-null  float64
 4   atemp       10886 non-null  float64
 5   humidity    10886 non-null  int64  
 6   windspeed   10886 non-null  float64
 7   count       10886 non-null  int64  
 8   mes         10886 non-null  object 
 9   weekday     10886 non-null  object 
 10  hora        10886 non-null  object 
dtypes: float64(3), int64(5), object(3)
memory usage: 935.6+ KB


In [ ]:
df['temp']=df['temp'].apply(lambda x: x*(9/5)+32)
df['atemp']=df['atemp'].apply(lambda x: x*(9/5)+32)
df['temp_variance']=df['atemp']-df['temp']

In [ ]:
df.head()

,holiday,workingday,weather,temp,atemp,humidity,windspeed,count,mes,weekday,hora,temp_variance
0,0,0,1,49.712,57.911,81,0.0,16,January,Saturday,00:00:00,8.199
1,0,0,1,48.236,56.543,80,0.0,40,January,Saturday,01:00:00,8.307
2,0,0,1,48.236,56.543,80,0.0,32,January,Saturday,02:00:00,8.307
3,0,0,1,49.712,57.911,75,0.0,13,January,Saturday,03:00:00,8.199
4,0,0,1,49.712,57.911,75,0.0,1,January,Saturday,04:00:00,8.199


##MODELO

Aplicamos modelo de regresión random forest

In [ ]:
X=df.drop(columns='count')
y=df['count']
X_train,X_test,y_train,y_test=train_test_split(X, y, random_state=42, test_size=0.3 )

In [ ]:
cat_sel=make_column_selector(dtype_include='object')
num_sel=make_column_selector(dtype_include='number')

scaler=StandardScaler()
ohe=OneHotEncoder(handle_unknown='ignore')
num_t=(scaler,num_sel)
cat_t=(ohe,cat_sel)

col_transformer = make_column_transformer(num_t, cat_t, remainder = 'passthrough')


In [ ]:
col_transformer.fit(X_train)
X_train_p=col_transformer.transform(X_train)
X_test_p=col_transformer.transform(X_test)


In [ ]:
rf=RandomForestRegressor(n_estimators=100, max_depth=None)
rf.fit(X_train_p, y_train)

RandomForestRegressor()

In [ ]:
#R2 del arbol por defecto
r2_train = rf.score(X_train_p, y_train)
r2_test = rf.score(X_test_p, y_test)
print('R2 train: ', r2_train)
print('R2_test: ', r2_test)

R2 train:  0.9762684747403368
R2_test:  0.831174692157783


R2 del modelo con el conjunto de prueba es 0.82